# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117657e+02     1.477491e+00
 * time: 0.09445786476135254
     1     1.102494e+01     8.954741e-01
 * time: 1.0199368000030518
     2    -1.164921e+01     1.002777e+00
 * time: 1.1654179096221924
     3    -3.397193e+01     8.434768e-01
 * time: 1.32082200050354
     4    -4.744727e+01     6.162960e-01
 * time: 1.482395887374878
     5    -5.694513e+01     2.069684e-01
 * time: 1.6342477798461914
     6    -5.985835e+01     1.572545e-01
 * time: 1.7464938163757324
     7    -6.096027e+01     5.607195e-02
 * time: 1.8670639991760254
     8    -6.138554e+01     6.460250e-02
 * time: 1.9882349967956543
     9    -6.164933e+01     4.898537e-02
 * time: 2.1010468006134033
    10    -6.186446e+01     2.496488e-02
 * time: 2.2211549282073975
    11    -6.201459e+01     1.965047e-02
 * time: 2.340460777282715
    12    -6.207534e+01     1.566714e-02
 * time: 2.4543638229370117
    13    -6.212796e+01     1.510421e-02
 * time: 2.5735838413238

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671085
    AtomicLocal         -18.8557704
    AtomicNonlocal      14.8522644
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336828

    total               -62.261666459592
